In [1]:
%reload_ext autotime
import pandas as pd
import plotly.io as pio
import plotly.express as px # Plotting
from tqdm import tqdm
tqdm.pandas()
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
pio.renderers.default = 'notebook'
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

time: 2.28 s (started: 2025-06-26 11:57:52 +12:00)


In [2]:
df = pd.read_excel("LLM_results_cleaned.xlsx")
df = df[(df.timeLeft == "closed") & (df.goal > 0)]
df.sentiment = df.sentiment.astype(str).str.split(" ").str[0].astype(float)
df.truth = df.truth.astype(str).str.split(" ").str[0].astype(float)
df.urgency = df.urgency.astype(str).str.split(" ").str[0].astype(float)
df["Target_completion"] = df.amountRaised / df.goal * 100
df

,Index,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri,start,end,n_questions,location,n_donors,description,use_of_funds,whos_involved,moderated,n_updates,updates,_21,text,condition,ICD10,ICD,name,gender,age,age_group,ethnicity,urgency,sentiment,truth,notes,smiling,deservingness,attractiveness,use,Target_completion,Charity
0,31,Helping a person in need,Helping out a person in need that would drop his hat to help someone else,https://static.givealittle.co.nz/assets/hero/ed760e80-7e10-4078-b7a8-b2ec0104419b-320:False,https://givealittle.co.nz/cause/helping-a-person-in-need,1700.0,2000.0,closed,Read more,https://givealittle.co.nz/cause/helping-a-person-in-need,29 May 2025,8 Jun 2025,0,"Blenheim, Marlborough",54.0,"Tok had a serious injury while running — he tore a muscle, which led to internal bleeding and ev...",medical care where needed and general stuff around the house if needed also and utility's,"Created by, and paying to a verified bank account of,\r\nMatthew bergman on behalf of Tok H...",Page Moderated,3.0,Big thanks\n7 June 2025\nHey team huge thanks to all that have donated Tok is humbled beyond wor...,NaN,Helping a person in need Helping a person in need Helping out a person in need that would drop h...,Muscle tear leading to internal bleeding and infection,S86.0,"Injury, poisoning and certain other consequences of external causes",Tok,male,35,15-64,Pacific Islander,90.0,70.0,95.0,Age estimated based on appearance; ethnicity inferred from image; sentiment reflects a mix of gr...,1,95,60,medical expenses,85.000000,False
1,56,Please help Walter 🙏 to get the veterinary care he needs.,Please help us 🙏 Walty needs bloodwork to rule out cancer 😪,https://static.givealittle.co.nz/assets/hero/a5763829-1835-4780-adc3-b2e500bd5557-320:False,https://givealittle.co.nz/cause/please-help-walter-to-get-the-veterinary-care-he,380.0,500.0,closed,Read more,https://givealittle.co.nz/cause/please-help-walter-to-get-the-veterinary-care-he,22 May 2025,1 Jun 2025,0,Otago,14.0,"Walter has been my life for 15 years, since he was a pup. It has always being just the two of us...","The money raised will go into Walts vet costsFull bloods, medication etc at the moment is quoted...","Created by, and paying to a verified bank account of,\r\nKerri Back",Page Moderated,1.0,Bloodwork done.\n23 May 2025\nThank you so much for your kind donations.\nWalty has had his bloo...,NaN,Please help Walter 🙏 to get the veterinary care he needs. Please help Walter 🙏 to get the veteri...,Pancreatitis,K85,11,Walter,male,15,15-64,Unknown,90.0,70.0,95.0,"The primary subject is an animal named Walter, not a human. The text mentions pancreatitis as th...",1,90,80,medical expenses,76.000000,False
2,59,Medical Treatment for Atish Lal for Coronary Artery Disease,medical Treatment,https://static.givealittle.co.nz/assets/hero/0a81d0ab-2bfc-4890-8049-b2e3017fea04-320:False,https://givealittle.co.nz/cause/medical-treatment-for-atish-lal-for-coronary,380.0,14900.0,closed,Read more,https://givealittle.co.nz/cause/medical-treatment-for-atish-lal-for-coronary,20 May 2025,6 Jun 2025,0,Auckland,6.0,"Dear Family and Friends,\nI hope this message finds you well.\nI am reaching out today with a hu...","Estimated costs for Treatment & Travel:- PTCA treatment package + home stay : $11,000 USD- Reser...",Created by\r\nShaneel Prasad \n\n\n\n\r\n Paying to a verified bank account of\r\nPriya Go...,Page Moderated,NaN,NaN,NaN,Medical Treatment for Atish Lal for Coronary Artery Disease Medical Treatment for Atish Lal for ...,Coronary Artery Disease,I25,I - Diseases of the circulatory system,Atish Lal,male,35,15-64,Indian,90.0,70.0,95.0,Age estimated based on typical age range for coronary artery disease; ethnicity inferred from th...,0,85,50,medical expenses,2.550336,False
3,74,Help me to reclaim my freedom.,Help my to reclaim my freedom and contribute to my community.,https://static.givealittle.co.nz/assets/hero/0e91c689-ef0e-4c5c-aa9d-b2e0010f721b-320:False,https://givealittl

time: 3.78 s (started: 2025-06-26 11:57:54 +12:00)


In [3]:
df.gender.value_counts().head(20)

gender
female    2258
male      1613
other      540
Name: count, dtype: int64

time: 9.54 ms (started: 2025-06-26 11:57:58 +12:00)


In [4]:
print("\n".join(df[df.gender == "Male and Female"].uri.to_list()))


time: 1.66 ms (started: 2025-06-26 11:57:58 +12:00)


In [5]:
df.gender = df.gender.str.lower().where(df.gender.str.lower().isin(['male', 'female']), 'other')
df.gender.value_counts()

gender
female    2258
male      1613
other      540
Name: count, dtype: int64

time: 11.8 ms (started: 2025-06-26 11:57:58 +12:00)


In [6]:
df.ethnicity.value_counts().head(20)

ethnicity
Caucasian                  2541
other                       396
Māori                       272
South Asian                 214
Asian                       161
Pacific Islander            144
Not specified               121
Unknown                     121
African                      57
Filipino                     54
Indian                       49
East Asian                   42
White                        39
Mixed                        38
South Asian/Indian           35
Filipino/Asian               33
Latin American/Hispanic      29
Fijian                       25
Hispanic/Latino              22
Not applicable               18
Name: count, dtype: int64

time: 6.9 ms (started: 2025-06-26 11:57:58 +12:00)


In [7]:
df.ethnicity = df.ethnicity.str.extract(r"(.+?)(?= \(|$)")
df.ethnicity.value_counts().head(20)

ethnicity
Caucasian                  2541
other                       396
Māori                       272
South Asian                 214
Asian                       161
Pacific Islander            144
Not specified               121
Unknown                     121
African                      57
Filipino                     54
Indian                       49
East Asian                   42
White                        39
Mixed                        38
South Asian/Indian           35
Filipino/Asian               33
Latin American/Hispanic      29
Fijian                       25
Hispanic/Latino              22
Not applicable               18
Name: count, dtype: int64

time: 14.3 ms (started: 2025-06-26 11:57:58 +12:00)


In [8]:
df.ethnicity = df.ethnicity.replace({
    "European/Caucasian": "Caucasian",
    "European/White": "Caucasian",
    "European/White": "Caucasian",
    "European/New Zealand": "Caucasian",
    "Maori": "Māori"
})
top_20 = df.ethnicity.value_counts().head(20)
display(top_20)
df.ethnicity = df.ethnicity.where(df.ethnicity.isin(top_20.index), "other")

ethnicity
Caucasian                  2541
other                       396
Māori                       272
South Asian                 214
Asian                       161
Pacific Islander            144
Not specified               121
Unknown                     121
African                      57
Filipino                     54
Indian                       49
East Asian                   42
White                        39
Mixed                        38
South Asian/Indian           35
Filipino/Asian               33
Latin American/Hispanic      29
Fijian                       25
Hispanic/Latino              22
Not applicable               18
Name: count, dtype: int64

time: 8.59 ms (started: 2025-06-26 11:57:58 +12:00)


In [9]:
df.condition.str.lower().value_counts().head(20)

condition
cancer                     192
gender dysphoria           135
breast cancer              135
stroke                      75
multiple sclerosis          65
endometriosis               45
bowel cancer                44
type 1 diabetes             39
melanoma                    38
obesity                     37
none specified              36
cerebral palsy              35
terminal cancer             33
infertility                 32
mental health issues        31
covid-19                    27
lung cancer                 27
coronary artery disease     27
hearing loss                26
brain tumor                 26
Name: count, dtype: int64

time: 8.06 ms (started: 2025-06-26 11:57:58 +12:00)


In [10]:
df.sentiment.value_counts()

sentiment
70.0     1717
80.0      603
50.0      524
60.0      493
90.0      422
30.0      258
85.0      235
20.0       64
95.0       35
40.0       28
75.0       26
10.0        5
100.0       1
Name: count, dtype: int64

time: 5.31 ms (started: 2025-06-26 11:57:58 +12:00)


In [11]:
df.use.str.lower().value_counts().head(20)

use
medical expenses                     3287
travel expenses                       678
experimental therapies                177
lost wages                            141
funeral expenses                       19
funeral costs                          10
medical expenses, travel expenses       5
other                                   4
educational tools                       3
educational expenses                    3
travel expenses, lost wages             3
living costs                            2
legal expenses                          2
financial expenses                      2
living expenses                         2
operational costs                       2
household expenses                      2
educational campaigns                   2
celebration                             1
renovation supplies                     1
Name: count, dtype: int64

time: 7.98 ms (started: 2025-06-26 11:57:58 +12:00)


In [12]:
df.age_group = df.age_group.str.extract(r"(.+?)(?= \(|$)")
df.age_group = df.age_group.str.lower().where(df.age_group.str.lower().isin(['0-14', '15-64', '65+']), 'indeterminate/unknown')
df.age_group.value_counts()

age_group
15-64                    2809
0-14                      649
65+                       558
indeterminate/unknown     395
Name: count, dtype: int64

time: 20 ms (started: 2025-06-26 11:57:58 +12:00)


In [13]:
df.gender.value_counts()

gender
female    2258
male      1613
other      540
Name: count, dtype: int64

time: 5.55 ms (started: 2025-06-26 11:57:58 +12:00)


In [14]:
df["Charity"] = df.whos_involved.str.contains(r"(Charity)", regex=False)

time: 3.6 ms (started: 2025-06-26 11:57:58 +12:00)


In [15]:
df.to_excel("LLM_results_cleaned.xlsx", index=False)

time: 5.98 s (started: 2025-06-26 11:57:58 +12:00)


In [16]:
df["met_target"] = (df.amountRaised >= df.goal).astype(int)

time: 2.18 ms (started: 2025-06-26 11:58:04 +12:00)


In [17]:
def parse(age):
    try:
        return float(age)
    except ValueError:
        return None

df["parsed_age"] = df.age.apply(parse)
df.parsed_age.value_counts().head(20)

parsed_age
35.0    371
30.0    247
25.0    189
45.0    186
60.0    136
50.0    128
55.0    117
65.0    111
40.0     95
28.0     79
20.0     72
38.0     61
23.0     45
17.0     42
18.0     41
8.0      40
22.0     39
13.0     39
10.0     37
70.0     36
Name: count, dtype: int64

time: 9.83 ms (started: 2025-06-26 11:58:04 +12:00)


In [18]:
df.met_target.value_counts()

met_target
0    3785
1     626
Name: count, dtype: int64

time: 4.16 ms (started: 2025-06-26 11:58:04 +12:00)


In [19]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, chi2_contingency, pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

def analyze_feature_influence(df, target_col='met_target'):
    results = {}
    target = df[target_col]
    is_target_numeric = np.issubdtype(target.dtype, np.number)

    for col in df.columns:
        if col == target_col:
            continue
        
        feature = df[col]
        if feature.isnull().any() or target.isnull().any():
            continue  # skip missing for simplicity

        if is_target_numeric:
            # Target is numeric
            if np.issubdtype(feature.dtype, np.number):
                # Pearson correlation
                corr, p_value = pearsonr(feature, target)
                results[col] = {
                    'test': 'pearsonr',
                    'stat': corr,
                    'p_value': p_value
                }
            else:
                # ANOVA (feature is categorical)
                groups = [target[feature == val] for val in feature.unique()]
                stat, p_value = f_oneway(*groups)
                results[col] = {
                    'test': 'anova',
                    'stat': stat,
                    'p_value': p_value
                }
        else:
            # Target is categorical
            if not np.issubdtype(feature.dtype, np.number):
                # Chi-squared test
                contingency = pd.crosstab(df[col], target)
                stat, p, _, _ = chi2_contingency(contingency)
                results[col] = {
                    'test': 'chi2',
                    'stat': stat,
                    'p_value': p
                }

    return pd.DataFrame(results).T.sort_values('p_value')

# Usage:
# df = pd.read_csv('your_data.csv')
df.met_target = df.met_target.astype(bool)
analyze_feature_influence(df, target_col='met_target')

,test,stat,p_value
age_group,chi2,36.113648,0.0
ethnicity,chi2,46.450561,0.000428
deservingness,chi2,35.895094,0.000616
end,chi2,2164.655553,0.254419
gender,chi2,2.247197,0.325108
start,chi2,2401.026199,0.382451
whos_involved,chi2,4353.107405,0.411837
title,chi2,4391.565606,0.469235
pitch,chi2,4402.788284,0.489563
text,chi2,4411.0,0.492921


time: 638 ms (started: 2025-06-26 11:58:04 +12:00)


In [20]:
formula = "met_target ~ parsed_age + gender + ethnicity"
logit = smf.glm(formula=formula, data=df, family=sm.families.Binomial(link=sm.families.links.Logit())).fit()
display(logit.summary())
display(f"AIC (Logit): {logit.aic:.2f}")
probit = smf.glm(formula=formula, data=df, family=sm.families.Binomial(link=sm.families.links.Probit())).fit()
display(probit.summary())
display(f"AIC (Probit): {probit.aic:.2f}")

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Generalized Linear Model Regression Results                             
=====================================================================================================
Dep. Variable:     ['met_target[False]', 'met_target[True]']   No. Observations:                 3144
Model:                                                   GLM   Df Residuals:                     3121
Model Family:                                       Binomial   Df Model:                           22
Link Function:                                         Logit   Scale:                          1.0000
Method:                                                 IRLS   Log-Likelihood:                -1211.1
Date:                                       Thu, 26 Jun 2025   Deviance:                       2422.2
Time:                                               11:58:05   Pearson chi2:                 3.04e+03
No. Iterations:                                           23   Pseudo R-squ. (CS):            0.03297
Covariance Type:                                   nonrobust                                         
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                1.2638      0.561      2.252      0.024       0.164       2.364
gender[T.male]                          -0.1609      0.109     -1.482      0.138      -0.374       0.052
gender[T.other]                          0.6620      0.317      2.088      0.037       0.041       1.284
ethnicity[T.Asian]                       0.7233      0.651      1.112      0.266      -0.552       1.998
ethnicity[T.Caucasian]                  -0.1281      0.553     -0.232      0.817      -1.211       0.955
ethnicity[T.East Asian]                 -0.5132      0.683     -0.751      0.452      -1.852       0.826
ethnicity[T.Fijian]                     22.7701   3.48e+04      0.001      0.999   -6.82e+04    6.82e+04
ethnicity[T.Filipino]                    1.8208      1.153      1.579      0.114      -0.439       4.080
ethnicity[T.Filipino/Asian]             22.6701   2.37e+04      0.001      0.999   -4.64e+04    4.65e+04
ethnicity[T.Hispanic/Latino]             0.3480      0.934      0.373      0.709      -1.482       2.178
ethnicity[T.Indian]                     22.7678   2.32e+04      0.001      0.999   -4.55e+04    4.56e+04
ethnicity[T.Latin American/Hispanic]     0.6336      0.926      0.684      0.494      -1.181       2.448
ethnicity[T.Mixed]                       1.0707      1.172      0.914      0.361      -1.226       3.367
ethnicity[T.Māori]                      -0.2056      0.584     -0.352      0.725      -1.350       0.939
ethnicity[T.Not applicable]             23.1661   7.56e+04      0.000      1.000   -1.48e+05    1.48e+05
ethnicity[T.Not specified]              -1.3939      1.414     -0.986      0.324      -4.166       1.378
ethnicity[T.Pacific Islander]            0.1529      0.625      0.245      0.807      -1.072       1.378
ethnicity[T.South Asian]                 1.0166      0.639      1.592      0.111      -0.235       2.268
ethnicity[T.South Asian/Indian]          1.6764      1.158      1.447      0.148      -0.594       3.947
ethnicity[T.Unknown]                     0.7715      0.762      1.012      0.312      -0.723       2.266
ethnicity[T.White]                       0.3683      0.924      0.399      0.690      -1.442       2.179
ethnicity[T.other]                       0.5103      0.593      0.860      0.390      -0.653       1.673
parsed_age                               0.0163      0.003      5.228      0.000       0.010       0.022
====================================================================================================

'AIC (Logit): 2468.19'

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Generalized Linear Model Regression Results                             
=====================================================================================================
Dep. Variable:     ['met_target[False]', 'met_target[True]']   No. Observations:                 3144
Model:                                                   GLM   Df Residuals:                     3121
Model Family:                                       Binomial   Df Model:                           22
Link Function:                                        Probit   Scale:                          1.0000
Method:                                                 IRLS   Log-Likelihood:                -1210.7
Date:                                       Thu, 26 Jun 2025   Deviance:                       2421.3
Time:                                               11:58:05   Pearson chi2:                 3.04e+03
No. Iterations:                                           22   Pseudo R-squ. (CS):            0.03324
Covariance Type:                                   nonrobust                                         
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                0.7899      0.311      2.537      0.011       0.180       1.400
gender[T.male]                          -0.0893      0.059     -1.502      0.133      -0.206       0.027
gender[T.other]                          0.3576      0.162      2.204      0.028       0.040       0.676
ethnicity[T.Asian]                       0.3624      0.351      1.033      0.302      -0.325       1.050
ethnicity[T.Caucasian]                  -0.0893      0.306     -0.291      0.771      -0.690       0.511
ethnicity[T.East Asian]                 -0.3151      0.385     -0.819      0.413      -1.069       0.439
ethnicity[T.Fijian]                      5.5143   4574.421      0.001      0.999   -8960.186    8971.214
ethnicity[T.Filipino]                    0.8597      0.526      1.634      0.102      -0.171       1.891
ethnicity[T.Filipino/Asian]              5.4792   3109.308      0.002      0.999   -6088.652    6099.610
ethnicity[T.Hispanic/Latino]             0.1732      0.500      0.347      0.729      -0.806       1.152
ethnicity[T.Indian]                      5.5565   3040.889      0.002      0.999   -5954.476    5965.589
ethnicity[T.Latin American/Hispanic]     0.3131      0.486      0.644      0.519      -0.639       1.266
ethnicity[T.Mixed]                       0.5161      0.577      0.895      0.371      -0.614       1.646
ethnicity[T.Māori]                      -0.1325      0.324     -0.409      0.683      -0.768       0.503
ethnicity[T.Not applicable]              5.6875      1e+04      0.001      1.000   -1.96e+04    1.96e+04
ethnicity[T.Not specified]              -0.8353      0.831     -1.005      0.315      -2.465       0.794
ethnicity[T.Pacific Islander]            0.0651      0.344      0.189      0.850      -0.609       0.739
ethnicity[T.South Asian]                 0.4915      0.341      1.443      0.149      -0.176       1.159
ethnicity[T.South Asian/Indian]          0.8286      0.548      1.512      0.130      -0.245       1.903
ethnicity[T.Unknown]                     0.4148      0.408      1.017      0.309      -0.385       1.214
ethnicity[T.White]                       0.1823      0.483      0.377      0.706      -0.764       1.129
ethnicity[T.other]                       0.2499      0.325      0.768      0.442      -0.388       0.888
parsed_age                               0.0091      0.002      5.374      0.000       0.006       0.012
====================================================================================================

'AIC (Probit): 2467.33'

time: 265 ms (started: 2025-06-26 11:58:05 +12:00)


In [21]:
df.met_target = df.met_target.astype(int)
smf.ols(formula="met_target ~ parsed_age + gender + ethnicity", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             met_target   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     4.075
Date:                Thu, 26 Jun 2025   Prob (F-statistic):           5.74e-10
Time:                        11:58:05   Log-Likelihood:                -1073.3
No. Observations:                3144   AIC:                             2193.
Df Residuals:                    3121   BIC:                             2332.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                0.2092      0.069      3.047      0.002       0.075       0.344
gender[T.male]                           0.0197      0.013      1.542      0.123      -0.005       0.045
gender[T.other]                         -0.0717      0.031     -2.278      0.023      -0.133      -0.010
ethnicity[T.Asian]                      -0.0722      0.074     -0.970      0.332      -0.218       0.074
ethnicity[T.Caucasian]                   0.0137      0.068      0.203      0.839      -0.119       0.146
ethnicity[T.East Asian]                  0.0744      0.088      0.845      0.398      -0.098       0.247
ethnicity[T.Fijian]                     -0.1465      0.113     -1.293      0.196      -0.369       0.076
ethnicity[T.Filipino]                   -0.1147      0.086     -1.332      0.183      -0.284       0.054
ethnicity[T.Filipino/Asian]             -0.1341      0.092     -1.464      0.143      -0.314       0.046
ethnicity[T.Hispanic/Latino]            -0.0412      0.105     -0.392      0.695      -0.247       0.165
ethnicity[T.Indian]                     -0.1442      0.091     -1.586      0.113      -0.322       0.034
ethnicity[T.Latin American/Hispanic]    -0.0654      0.099     -0.661      0.509      -0.260       0.129
ethnicity[T.Mixed]                      -0.1012      0.109     -0.931      0.352      -0.314       0.112
ethnicity[T.Māori]                       0.0248      0.072      0.346      0.730      -0.116       0.165
ethnicity[T.Not applicable]             -0.1937      0.209     -0.928      0.354      -0.603       0.216
ethnicity[T.Not specified]               0.2226      0.209      1.063      0.288      -0.188       0.633
ethnicity[T.Pacific Islander]           -0.0211      0.075     -0.280      0.779      -0.169       0.127
ethnicity[T.South Asian]                -0.0866      0.072     -1.208      0.227      -0.227       0.054
ethnicity[T.South Asian/Indian]         -0.1275      0.093     -1.369      0.171      -0.310       0.055
ethnicity[T.Unknown]                    -0.0819      0.085     -0.968      0.333      -0.248       0.084
ethnicity[T.White]                      -0.0360      0.097     -0.371      0.711      -0.226       0.154
ethnicity[T.other]                      -0.0546      0.071     -0.773      0.440      -0.193       0.084
parsed_age                              -0.0019      0.000     -5.296      0.000      -0.003      -0.001
==============================================================================
Omnibus:                     1097.203   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2724.193
Skew:                           1.997   Prob(JB):                         0.00
Kurtosis:                       5.199

time: 73.1 ms (started: 2025-06-26 11:58:05 +12:00)


In [22]:
smf.ols(formula="met_target ~ sentiment + smiling + gender + truth", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             met_target   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     16.83
Date:                Thu, 26 Jun 2025   Prob (F-statistic):           1.64e-16
Time:                        11:58:05   Log-Likelihood:                -1573.4
No. Observations:                4411   AIC:                             3159.
Df Residuals:                    4405   BIC:                             3197.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.2053      0.119     -1.719      0.086      -0.439       0.029
gender[T.male]      0.0238      0.011      2.080      0.038       0.001       0.046
gender[T.other]    -0.0248      0.017     -1.451      0.147      -0.058       0.009
sentiment           0.0024      0.000      6.648      0.000       0.002       0.003
smiling             0.0303      0.011      2.777      0.006       0.009       0.052
truth               0.0018      0.001      1.300      0.194      -0.001       0.004
==============================================================================
Omnibus:                     1523.630   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3750.469
Skew:                           1.994   Prob(JB):                         0.00
Kurtosis:                       5.121   Cond. No.                     2.62e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

time: 43.6 ms (started: 2025-06-26 11:58:05 +12:00)


In [23]:
smf.ols(formula="met_target ~ sentiment + smiling + gender + truth + urgency + ethnicity", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             met_target   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     5.090
Date:                Thu, 26 Jun 2025   Prob (F-statistic):           2.11e-15
Time:                        11:58:05   Log-Likelihood:                -1552.0
No. Observations:                4411   AIC:                             3156.
Df Residuals:                    4385   BIC:                             3322.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -0.1561      0.130     -1.198      0.231      -0.412       0.099
gender[T.male]                           0.0278      0.012      2.415      0.016       0.005       0.050
gender[T.other]                         -0.0305      0.019     -1.602      0.109      -0.068       0.007
ethnicity[T.Asian]                      -0.0423      0.053     -0.793      0.428      -0.147       0.062
ethnicity[T.Caucasian]                   0.0026      0.047      0.056      0.955      -0.089       0.094
ethnicity[T.East Asian]                  0.0371      0.070      0.528      0.597      -0.101       0.175
ethnicity[T.Fijian]                     -0.0329      0.083     -0.397      0.692      -0.195       0.130
ethnicity[T.Filipino]                   -0.1162      0.066     -1.771      0.077      -0.245       0.012
ethnicity[T.Filipino/Asian]             -0.1576      0.076     -2.082      0.037      -0.306      -0.009
ethnicity[T.Hispanic/Latino]            -0.0530      0.087     -0.609      0.542      -0.224       0.118
ethnicity[T.Indian]                     -0.0817      0.067     -1.210      0.226      -0.214       0.051
ethnicity[T.Latin American/Hispanic]    -0.0572      0.079     -0.726      0.468      -0.212       0.097
ethnicity[T.Mixed]                      -0.1183      0.073     -1.631      0.103      -0.261       0.024
ethnicity[T.Māori]                      -0.0146      0.050     -0.290      0.772      -0.113       0.084
ethnicity[T.Not applicable]              0.0305      0.094      0.324      0.746      -0.154       0.215
ethnicity[T.Not specified]               0.0111      0.057      0.195      0.845      -0.101       0.123
ethnicity[T.Pacific Islander]           -0.0385      0.054     -0.711      0.477      -0.145       0.068
ethnicity[T.South Asian]                -0.0960      0.052     -1.856      0.063      -0.197       0.005
ethnicity[T.South Asian/Indian]         -0.1322      0.074     -1.781      0.075      -0.278       0.013
ethnicity[T.Unknown]                    -0.0304      0.056     -0.545      0.586      -0.140       0.079
ethnicity[T.White]                      -0.0436      0.072     -0.605      0.545      -0.185       0.098
ethnicity[T.other]                      -0.0306      0.049     -0.625      0.532      -0.127       0.065
sentiment                                0.0025      0.000      6.442      0.000       0.002       0.003
smiling                                  0.0187      0.011      1.628      0.104      -0.004       0.041
truth                                    0.0015      0.001      1.057      0.291      -0.001       0.004
urgency                                 -0.0001      0.001     -0.223      0.823      -0.001       0.001
======================================

time: 74.8 ms (started: 2025-06-26 11:58:05 +12:00)


In [24]:
model = RandomForestRegressor()
cols = "sentiment + smiling + gender + truth + urgency + ethnicity".split(" + ")
X = pd.get_dummies(df[cols])
y = df["met_target"]
model.fit(X, y)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


time: 529 ms (started: 2025-06-26 11:58:05 +12:00)


In [25]:
list(zip(cols, model.feature_importances_))

[('sentiment', np.float64(0.20944326354630083)),
 ('smiling', np.float64(0.10208651345151197)),
 ('gender', np.float64(0.10317023310735184)),
 ('truth', np.float64(0.206018616591242)),
 ('urgency', np.float64(0.050877138523660895)),
 ('ethnicity', np.float64(0.05290824903974063))]

time: 11.7 ms (started: 2025-06-26 11:58:06 +12:00)
